In [ ]:
#

# LIBRARIES

import pandas as pd
import numpy as np
import itertools as it
from datetime import datetime
import networkx as nx
import math
import random
import os
import operator
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

from operator import itemgetter
from itertools import groupby
from sklearn.model_selection import train_test_split

In [ ]:
# CONSTANTS 

# DATASET_SIZE = 100000
# DATASET_SIZE =  15954 # most popular movies 200 && users que han interactuado con 50%
# DATASET_SIZE =  34030 # most popular movies 200 && users que han interactuado con 25%
DATASET_SIZE =  42482 # most popular movies 200 && users que han interactuado con 12.5%

# HALF_DATASET_SIZE = int(90*DATASET_SIZE / 100)
# HALF_DATASET_SIZE = int(75*DATASET_SIZE / 100)
# SECOND_HALF_DATASET = int(DATASET_SIZE - HALF_DATASET_SIZE)
RATING_THRESHOLD = 4
WEIGHT_THRESHOLD = 5
# K = 10
K = 3
range_K = [3,5,10]
MEASURES = ['aa', 'cn', 'ew', 'jn', 'pa', 'waa', 'wcn', 'wpa']
USERS_EVAL = 25

In [ ]:
def compareNodes(f_list, s_list):
    """
        Function that returns the number of users that have interact with both items
        Funcion que devuelve el numero de usuarios que han interactuado con ambos items
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos, threshold):
    """
        Function that creates graph links with the information about the set. The weight has to be grater or equal to threshold.
        
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion. El peso tiene que ser mayor o igual al umbral.
        
        Format of links list -> [(Node1, Node2, weight), ......]
    """
    resultado = list()  
    
    # hago todas las posibles combinaciones de problemas
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de usuarios que ha hecho cada problema
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= threshold:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado

In [ ]:
def create_graph_nx(list_nodes, list_links):
    """
        Function that creates a graph with the format from NetworkX 
        
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo

In [ ]:
def nodes_connected(u, v, graph):
    return u in graph.neighbors(v)

In [ ]:
def areAccessible(relevant, possible, graph):
    access = list()
    for r in relevant:
        for p in possible:
            if nodes_connected(r, p, graph):
                access.append(r)
    return list(set(access))

In [ ]:
def apply_aa(i1, i2, graph):
    """
        Funcion que devuelve para cada par de nodos, el sumatorio de 1/log(N(z)), siendo N(z) el grado del nodo z para todo z 
        perteneciente al conjunto de nodos en comun de ese par de nodos
    """
    
    # obtengo un iterador de un solo elemento que tiene en la tercera posicion el valor de AA para el par de nodos
    value = nx.adamic_adar_index(graph, [(i1, i2)])
    
    value_aa = 0
    for u, v, p in value:
        # itero el iterador, guardando el valor de adar adamic
        value_aa = p
    
    return value_aa

In [ ]:
def apply_cn(i1, i2, graph):
    """
        Funcion que devuelve el numero de vecinos en comun de esos dos nodos
    """
    return len(list(nx.common_neighbors(graph, i1, i2)))

In [ ]:
def apply_ew(fst, snd, graph):
    """
        Funcion que devuelve el peso del enlace en cada par
    """
    
    weight = graph.get_edge_data(fst, snd)
    
    # print(weight)
    
    if weight == None: # devuelve 0 en caso de que no exista enlace
        return 0
    else: # si si existe, devuelve el peso
        return weight['weight']

In [ ]:
def apply_jn(i1, i2, graph):
    """
        Funcion que devuelve el numero de vecinos en comun de esos dos nodos
    """
    values_jn = nx.jaccard_coefficient(graph, [(i1, i2)])
    
    value_jn = 0
    for u, v, p in values_jn:
        value_jn = p # saco el valor
        
    return value_jn

In [ ]:
def apply_pa(i1, i2, graph):
    """
        Funcion que devuelve el valor de preferential attachment
    """
    values_pa = nx.preferential_attachment(graph, [(i1, i2)])
    
    value_pa = 0
    for u, v, p in values_pa:
        value_pa = p # saco el valor
        
    return value_pa


In [ ]:
def apply_waa(i1, i2, graph):
    """
        Funcion que devuelve para cada par de nodos, el valor de weighted adar adamic
    """
    
    # primero tengo que calcular los common neighbors de ambos items
    cn_list = nx.common_neighbors(graph, i1, i2)
    
    # ahora tengo que hacer el sumatorio del valor para cada elemento de cn_list
    value_waa = sum([((graph[i1][x]['weight'] + graph[i2][x]['weight']) / math.log(1 + graph.degree(x, weight="weight"), 10) )  for x in cn_list])    
    
    
    return value_waa

In [ ]:
def apply_wcn(i1, i2, graph):
    """
        Funcion que devuelve el numero de vecinos en comun de esos dos nodos
    """
    cn_list = nx.common_neighbors(graph, i1, i2)
    
    value_wcn = sum([graph[i1][x]['weight'] + graph[i2][x]['weight'] for x in cn_list])
    
    return value_wcn

In [ ]:
def apply_wpa(i1, i2, graph):
    """
        Funcion que devuelve el valor de weighted preferential attachment
    """
    value_wpa = graph.degree(i1, weight="weight") * graph.degree(i2, weight="weight")
        
    return value_wpa

In [ ]:
def apply_measure(i1, i2, graph, measure):
     
    """
        Function that resturn similarity value with the measure
        
    """
    # Aplico la funcion a cada fila
    if measure == 'aa':
        sim_result = apply_aa(i1, i2, graph)
    elif measure == 'cn':
        sim_result = apply_cn(i1, i2, graph)
    elif measure == 'ew':
        sim_result = apply_ew(i1, i2, graph)
    elif measure == 'jn':    
        sim_result = apply_jn(i1, i2, graph)
    elif measure == 'pa':    
        sim_result = apply_pa(i1, i2, graph)

    elif measure == 'waa':
        sim_result = apply_waa(i1, i2, graph)
    elif measure == 'wcn':    
        sim_result = apply_wcn(i1, i2, graph)
    elif measure == 'wpa':    
        sim_result = apply_wpa(i1, i2, graph)
        
    return sim_result

In [ ]:
def build_matrix_items(items, items_eval, graph, user):
    """ en cada fila, voy a tener el conjunto de items a evaluar
        en las columnas, todos los items del grafo"""
    
    if not os.path.exists('similarities/item_weigh/user_' + str(user) + '/'):
        os.makedirs('similarities/item_weigh/user_' + str(user) + '/')
                          
    for measure in MEASURES:
        with open('similarities/item_weigh/user_' + str(user) + '/item_' + measure + '.csv', 'w') as result_file:
            print('item1,item2,similarity', file=result_file)

            for i1 in items_eval:
                for i2 in items:
                    sim = apply_measure(i1, i2, graph, measure)
                    print(f"{i1},{i2},{sim}", file=result_file)

In [ ]:
def getting_sim_matrix(user):
    """ Read the arrays"""
    pd_sim_aa = pd.read_csv('similarities/item_weigh/user_' + str(user) + '/item_aa.csv')
    pd_sim_cn = pd.read_csv('similarities/item_weigh/user_' + str(user) + '/item_cn.csv')
    pd_sim_ew = pd.read_csv('similarities/item_weigh/user_' + str(user) + '/item_ew.csv')
    pd_sim_jn = pd.read_csv('similarities/item_weigh/user_' + str(user) + '/item_jn.csv')
    pd_sim_pa = pd.read_csv('similarities/item_weigh/user_' + str(user) + '/item_pa.csv')
    pd_sim_waa = pd.read_csv('similarities/item_weigh/user_' + str(user) + '/item_waa.csv')
    pd_sim_wcn = pd.read_csv('similarities/item_weigh/user_' + str(user) + '/item_wcn.csv')
    pd_sim_wpa = pd.read_csv('similarities/item_weigh/user_' + str(user) + '/item_wpa.csv')


    sim_aa = pd_sim_aa.pivot(index='item1', columns='item2', values='similarity')
    sim_cn = pd_sim_cn.pivot(index='item1', columns='item2', values='similarity')
    sim_ew = pd_sim_ew.pivot(index='item1', columns='item2', values='similarity')
    sim_jn = pd_sim_jn.pivot(index='item1', columns='item2', values='similarity')
    sim_pa = pd_sim_pa.pivot(index='item1', columns='item2', values='similarity')
    sim_waa = pd_sim_waa.pivot(index='item1', columns='item2', values='similarity')
    sim_wcn = pd_sim_wcn.pivot(index='item1', columns='item2', values='similarity')
    sim_wpa = pd_sim_wpa.pivot(index='item1', columns='item2', values='similarity')
    
    return sim_aa, sim_cn, sim_ew, sim_jn, sim_pa, sim_waa, sim_wcn, sim_wpa

In [ ]:
def write_recommendations(user, recom_list, measure):        
    f = open('recommendations/item_weighted-vot_recoms.csv', 'a')
    f.write(str(user) + ',' + measure + ',' + str(recom_list) + '\n') 
    
    f.close()

In [ ]:
def getKrecommendations(item_recommend, list_user_items, measure, items_eval):
    """
        Funcion que devuelve la lista de k mejores problemas para el usuario dado teniendo en cuenta que 
        las recomendaciones no son items con los que haya interactuado el usuario
    """
    
    # Para row[item], el cual es el index de la columna en la matriz, encontrar los k más similares con los que no haya 
    # interactuado el usario todavia
    # item_recommend = row['item']
    
    list_recom = dict()
    i = 0
    for elem in measure[item_recommend]:
        if items_eval[i] not in list_user_items:
            list_recom[items_eval[i]] = elem
        i = i + 1
    
    return list(list_recom.items())

In [ ]:
def getWeighing(item, items_recom_with_values):
    weight = sum([value for (it, value) in items_recom_with_values if it == item])
    
    return weight

In [ ]:
def delRepetitions(lista):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in lista if not (x in conjunto_vacio or function_add(x))]

In [ ]:
def getListRecom(df_users, k, measure, items_eval, user):
    list_recommendations = list()
    for item_recommend in df_users[user]:
        # Esto va a contener una lista de tuplas del tipo (item-recomendar, valor similitud), donde el item puede aparecer varias veces repetido
        list_recommendations = list_recommendations + getKrecommendations(item_recommend, df_users[user], measure, items_eval)

    # Contiene la suma de todos los valroes de similitud
    total = sum([x for (_,x) in list_recommendations])

    # Esto va a contener solo los items a recomendar sin repetición y sin valores
    items_recom_no_values = list(set([item for (item, _) in list_recommendations]))
    
    # Sistema de votación ponderada: para cada item que aparezca, sumar todos sus valores de similitud asociado / total
    items_recom = [(item, getWeighing(item, list_recommendations)/total) for item in items_recom_no_values]
    
    items_recom.sort(key=operator.itemgetter(1), reverse=True)
    
    # y me quedo con el primer elemento de la tupla, que es el item a recomendar
    list_sim_final = [x for (x,_) in items_recom] 
        
    # ahora elimino los items que estan en la lista de items con los que ha interactuado el target user
    list_final = [x for x in list_sim_final if x not in df_users[user]]
    
    # y quito las repeticiones
    list_final = delRepetitions(list_final)
    
    list_fin_rec = [x for x in list_final if x in items_eval]
    
    return list_fin_rec

In [ ]:
def apply_getKrecommendations(df_new, df_users, k, measure, items_eval, user, my_measure):
    """
    Function to generate a new column with the list of recommendations for each user
    """
    
    df_new['list_recommendations_original'] = df_new.apply(lambda row: getListRecom(df_users, k, measure, items_eval, user), axis=1)
    df_new['list_recommendations'] = df_new.apply(lambda row: row['list_recommendations_original'][:k], axis=1)    
        
    if k == 10:
        df_new.apply(lambda row: write_recommendations(user, row['list_recommendations_original'], my_measure), axis=1)
        
        
    return df_new

In [ ]:
def calculateMetricsResults(list_recom_items, list_recom_items_original, user_list_to_recommend, list_eval_items, list_rel_accessible, k):    
    set_df_metric = {'user_id': user_list_to_recommend, 'eval_items': list_eval_items, 'recom_items': list_recom_items, 'rel_accessible': list_rel_accessible, 'recom_items_original': list_recom_items_original}
    metric_df = pd.DataFrame.from_dict(set_df_metric)
    return metric_df

In [ ]:
def one_hit(row):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuario dado, ese usuario ha interactuado 
        con al menos uno de los items que se le ha recomendado en el evaluation_set. 
        Cero si no hay ningun item de los recomendados con los que el usuario haya interactuado
    """
    num_items_common = np.intersect1d(row['recom_items'], row['eval_items'])
    
    if len(num_items_common) >= 1:
        return 1
    else:
        return 0

In [ ]:
def mrr(row): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """

    num_items_common = np.intersect1d(row['recom_items'], row['eval_items'])
    
    if len(num_items_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        ranki = 0
        #print(ranki)
        while (i < len(row['recom_items'])) and (encontrado == False):
            if row['recom_items'][i] in row['eval_items']:
                # fst_correct_item = row['recom_items'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
                #print("entro")
            else:
                i = i + 1
                
        if ranki == 0:
            return 0
        else:
            return (1/ranki)

    else:
        return 0


In [ ]:
def precision(row):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los interactuados con el usuario estan entre los recomendados) / todos los recomendados
    """
    
    num_items_common = np.intersect1d(row['recom_items'], row['eval_items'])
    
    # print(num_items_common)
    
    return (len(num_items_common)/len(row['recom_items']))

In [ ]:
def recall(row):
    """
        Funcion que implementa la metrica recall
        (cuantos de los interactuados con el usuario estan entre los recomendados) / todos los evaluados
    """
    num_items_common = np.intersect1d(row['recom_items'], row['eval_items'])
    
    # print(num_items_common)
    
    return (len(num_items_common)/len(row['eval_items']))

In [ ]:
def f1(row):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    denominador = row['precision'] + row['recall']
    
    if denominador == 0:
        return 0
    else:
        return (2 * row['precision'] * row['recall']) / denominador

In [ ]:
def rPrecision(row):
    """
        Funcion que va a implementar la metrica r-precision: cuales de los recomendados 
        son relevantes en el conjunto de accesibles por el grafo 
    """
    if len(row['rel_accessible']) == 0:
        return 0
    else:
        recomendations = row['recom_items_original'][:len(row['rel_accessible'])]
        num_items_common = np.intersect1d(recomendations, row['rel_accessible'])

        # print(num_items_common)

        return (len(num_items_common)/len(row['rel_accessible']))

In [ ]:
def calculateScoreResults(metric_df):
    """
        Function to build a dataframe with the results for the evaluation metrics
    """
    metric_df['one_hit'] = metric_df.apply(lambda row: one_hit(row), axis=1)
    metric_df['mrr'] = metric_df.apply(lambda row: mrr(row), axis=1)
    metric_df['precision'] = metric_df.apply(lambda row: precision(row), axis=1)
    metric_df['recall'] = metric_df.apply(lambda row: recall(row), axis=1)
    metric_df['f1'] = metric_df.apply(lambda row: f1(row), axis=1)
    metric_df['rprec'] = metric_df.apply(lambda row: rPrecision(row), axis=1)

    result_one_hit = metric_df['one_hit'].mean()
    result_precision = metric_df['precision'].mean()
    result_mrr = metric_df['mrr'].mean()
    result_recall = metric_df['recall'].mean()
    result_f1 = metric_df['f1'].mean()
    result_rprec = metric_df['rprec'].mean()

    # voy a crear un diccionario con los resultados
    results_metrics = {'one_hit': result_one_hit, 'precision': result_precision, 'mrr': result_mrr, 'recall': result_recall, 'f1': result_f1, 'rprec': result_rprec}
    # results_metrics = {'rprec': result_rprec}

    
    return results_metrics

In [ ]:
def main_process(user, items_eval):
    #Conjunto con todas las interacciones de los usuarios, fueron extraidas las interacciones del usuario
    # objetivo con los items bajo evaluación.
    training_set = pd.read_csv('user-datasets/user_' + str(user) + '_training.csv')

    # Conjunto de interacciones del usuario objetivo con los productos seleccionados para evaluar.
    evaluation_set = pd.read_csv('user-datasets/user_' + str(user) + '_test.csv')

    # I get the list of nodes
    nodes = training_set.item.unique()
    print(len(nodes))

    # I create a dictionary: keys are the items, and values are the list of users that are interacted with this item
    grouped = training_set.groupby('item')['user'].apply(list)

    # I create the links with the suitable format for nx
    links = createLinks(grouped, nodes, WEIGHT_THRESHOLD)
    print(len(links))

    ##################### GRAPH CREATION
    # I create the graph
    graph = create_graph_nx(nodes, links)
    
    # diccionario que va a contener como key el user, como value, los items con los que ha interactuado el user
    df_users_simple = {}
    grouped_user = training_set.groupby('user')['item'].apply(list)
    for i,j in zip(grouped_user.index.tolist(), grouped_user.values.tolist()):
        df_users_simple[i] = j 
        
    # PARA LA CONSTRUCCION DE R PRECISION -----------
    grouped_user_eval = evaluation_set.groupby('user')['item'].apply(list)

    # convierto la serie en un dataframe
    df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_item_id':grouped_user_eval.values})

    user_list_to_recommend = list(evaluation_set.user.unique())

    # hago el filtro para los usuarios a los que tengo que recomendar
    df_users_eval = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
    # primero voy a ordenar la lista de usuarios a recomendar
    user_list_to_recommend.sort()
    list_eval_items = df_users_eval['list_item_id'].tolist()

    df_users_eval["films_watched"] = df_users_eval.apply (lambda row: df_users_simple[row['user_id']], axis=1)
    df_users_eval["rel_accessible"] = df_users_eval.apply (lambda row: areAccessible(row['list_item_id'], row['films_watched'], graph), axis=1)
    df_users_eval["num_accessible"] = df_users_eval.apply (lambda row: len(row['rel_accessible']), axis=1)
    list_rel_accessible = df_users_eval['rel_accessible'].tolist()
    list_eval_items = df_users_eval['list_item_id'].tolist()
    user_list_to_recommend = df_users_eval['user_id'].tolist()
    
    
    # construccion de matrices de similitud
    """
        items: Son todos los id de los productos que forman parte del conjunto de los 200 productos mas populares con el que el 12% de los usuarios han interactuado
        items_eval: Son el 20% de los productos seleccionados aleatoriamente del 100% del conjunto de productos (12% de las peliculas mas importantes)
        graph: es el grafo con las interacciones del conjunto del 12% de las peliculas mas populares.
        user: Es el usuario que se esta evaluando
    """
    build_matrix_items(items, items_eval, graph, user)

    sim_aa, sim_cn, sim_ew, sim_jn, sim_pa, sim_waa, sim_wcn, sim_wpa = getting_sim_matrix(user)
    list_measures = [sim_aa, sim_cn, sim_ew, sim_jn, sim_pa, sim_waa, sim_wcn, sim_wpa]

    # convierto la serie en un dataframe
    df_users = pd.DataFrame({'user_id':grouped_user.index, 'list_item_id':grouped_user.values})
    
    df_new = df_users[df_users['user_id'].isin(user_list_to_recommend)]
#     df_new = df_users.groupby('user_id').list_item_id.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)
#     df_new.columns = ['user_id','item']
    
    dataframe_k_measures_original = list()

    # df_new: contiene el usuario al cual se le haran las recomendaciones, este dataframe contiene una columna con todos los productos con los que
    # ha interactuado
    # df_users_simple: es un diccionario que contiene todos los usuarios, cuyo value es una lista de productos con los que han interactuado
    # k: Es el tamaño de la lista de recomendación
    # items_eval: Es la lista del 10% de elementos que fueron seleccionados para evaluar
    #
    dataframe_k_measures_original = [[apply_getKrecommendations(df_new, df_users_simple, k, list_measures[MEASURES.index(measure)], items_eval, user, measure).copy() for measure in MEASURES] for k in range_K]
    
    # dataframe_k_measures_original Es una matrix que se organiza de la siguiente manera dataframe_k_measures_original[k][measure], donde cada posición tiene un dataframe con las recomendaciones correspondientes
    """
        el metodo calculateMetricsResults recibe:
        1. dataframe_k_measures_original[k][MEASURES.index(measure)]['list_recommendations'].tolist(): Lista de recomendaciones de tamaño k
        2. dataframe_k_measures_original[k][MEASURES.index(measure)]['list_recommendations_original'].tolist(): Lista de recomendaciones con todos los elementos a recomendar
        3. user_list_to_recommend: Lista de usuarios a recomendar
        4. list_eval_items: Lista de productos de evaluación (Son el 20% de los productos extraidos del dataset original)
        5. list_rel_accessible: La lista de productos accesibles en base a los productos con los que ha interactuado el usuario target, y los
            productos que estan en el dataset de evaluación del usuario (Interacciones retiradas del dataset de entrenamiento). La accesibilidad
            de un producto esta basada en si los productos son vecinos en el grafo de interacción
        6. range_K[k]: Tamaño de K
    """
    metrics_results = [[calculateMetricsResults(dataframe_k_measures_original[k][MEASURES.index(measure)]['list_recommendations'].tolist(), dataframe_k_measures_original[k][MEASURES.index(measure)]['list_recommendations_original'].tolist(), user_list_to_recommend, list_eval_items, list_rel_accessible, range_K[k]) for measure in MEASURES] for k in range(K)]

    return metrics_results

In [ ]:
def write_results_file(dir, result, k, measure):
    f = open(dir, 'a')
    f.write(str(k) + ',' + measure + ',' + str(result['one_hit']) + ',' + str(result['precision']) + ',' + str(result['mrr']) + ',' + str(result['recall']) + ',' +  str(result['f1']) + ',' +  str(result['rprec']) + '\n') 
    
    f.close()

In [ ]:
df = pd.read_csv('most_pop_200_users_12.csv')


print(df)

In [ ]:
df = df.sort_values('timestamp')

print(df)

users = df.user.unique()
items = df.item.unique()
num_items = len(items)
num_users = len(users)

print(num_items)
print(num_users)

In [ ]:
items_eval = sorted([79, 154, 275, 137, 655, 692, 568, 235, 98, 284, 274, 204, 603, 203, 435, 245, 385, 472, 4])
len(items_eval)

In [ ]:
users = [1, 2, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 20, 21, 22, 23, 24, 25, 26, 28, 30, 32, 37, 38, 41, 42, 43, 44, 45, 48, 49, 52, 54, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 81, 82, 83, 84, 85, 87, 89, 90, 91, 92, 94, 95, 96, 97, 99, 101, 102, 103, 104, 106, 108, 109, 110, 113, 114, 115, 116, 117, 118, 119, 121, 123, 125, 128, 130, 135, 137, 138, 141, 144, 145, 148, 151, 152, 154, 157, 158, 159, 160, 161, 162, 164, 168, 174, 175, 177, 178, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 193, 194, 195, 197, 198, 200, 201, 207, 210, 213, 214, 215, 216, 217, 218, 221, 222, 223, 224, 226, 227, 230, 232, 233, 234, 235, 236, 237, 239, 243, 244, 246, 248, 249, 250, 251, 253, 254, 255, 256, 259, 262, 263, 264, 265, 267, 268, 269, 270, 271, 272, 274, 275, 276, 277, 279, 280, 283, 286, 287, 288, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 301, 303, 305, 307, 308, 311, 312, 313, 314, 315, 316, 318, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 332, 334, 336, 338, 339, 340, 342, 343, 344, 345, 346, 347, 348, 350, 352, 354, 357, 360, 361, 363, 365, 370, 371, 372, 373, 374, 378, 379, 380, 381, 382, 383, 385, 387, 388, 389, 391, 392, 393, 394, 395, 396, 397, 398, 399, 401, 402, 403, 405, 406, 407, 409, 411, 412, 413, 416, 417, 421, 422, 423, 424, 425, 426, 429, 430, 432, 433, 435, 436, 437, 442, 445, 447, 449, 450, 452, 453, 454, 455, 456, 457, 458, 459, 460, 463, 464, 465, 466, 468, 470, 472, 474, 476, 478, 479, 480, 481, 483, 484, 486, 487, 488, 489, 490, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 503, 504, 505, 506, 507, 508, 514, 518, 521, 523, 524, 525, 526, 527, 528, 530, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 545, 548, 551, 552, 553, 554, 555, 556, 559, 560, 561, 562, 566, 567, 568, 569, 573, 576, 577, 579, 582, 586, 587, 588, 590, 591, 592, 593, 595, 600, 601, 603, 605, 606, 608, 610, 615, 617, 618, 619, 620, 621, 622, 623, 624, 625, 627, 629, 630, 632, 633, 634, 637, 638, 639, 640, 642, 643, 645, 647, 648, 650, 653, 654, 655, 658, 659, 660, 661, 663, 664, 665, 666, 667, 668, 669, 670, 671, 674, 676, 677, 679, 680, 682, 683, 684, 686, 690, 692, 693, 694, 697, 698, 699, 703, 704, 705, 707, 708, 709, 710, 711, 712, 714, 715, 716, 717, 719, 721, 724, 727, 731, 733, 734, 735, 738, 739, 741, 745, 746, 747, 748, 749, 751, 753, 756, 757, 758, 759, 761, 763, 764, 766, 768, 770, 771, 773, 774, 776, 777, 778, 779, 780, 781, 782, 786, 788, 790, 793, 795, 796, 798, 802, 804, 805, 806, 807, 815, 821, 823, 825, 826, 829, 830, 831, 833, 834, 835, 836, 837, 838, 839, 840, 843, 844, 846, 847, 848, 850, 851, 852, 854, 860, 862, 864, 865, 867, 868, 870, 871, 872, 875, 877, 878, 880, 881, 882, 883, 885, 886, 887, 889, 890, 892, 893, 894, 896, 897, 899, 901, 902, 903, 905, 907, 908, 910, 911, 912, 913, 916, 918, 919, 921, 922, 923, 924, 927, 929, 930, 931, 932, 933, 934, 935, 936, 938, 939, 940, 942, 943]

In [ ]:
results = list()
i = 1
for user in users:
    print("Current user: " + str(i) + "---" + str(user) + "------")
    results.append(main_process(user, items_eval))
    i = i + 1

In [ ]:
metrics_results_ev = list()
for k in range(K):
    list_k = list()
    for measure in range(len(MEASURES)):
        # Esta iterando en results para obtener el dataframe de recomendaciones para "k" para
        # cada uno de los usuarios. cada elemento de results es una matriz con la forma results[k][measure], donde cada posición es un dataframe
        # con las recomendaciones.
        my_pd_metric = [df[k][measure] for df in results] 
        list_k.append(pd.concat(my_pd_metric))
    metrics_results_ev.append(list_k)

    # La estructura de my_pd_metric es [df1, df2], donde cada dataframe corresponde a cada usario, y para cada k y measure especifico
    # La estructura de list_k corresponde a [my_pd_metric_1=concat([df1, df2]), my_pd_metric_2=concat([df1, df2])] 
    # donde cada sub lista corresponde a una measure distinto
    # metrics_results_ev tiene la siguiente estructura: [[list_k_1, list_k_2], [list_k_1, list_k_2]] donde cada sub lista corresponde a un "k"
    # especifico

In [ ]:
metrics_results = [[calculateScoreResults(metrics_results_ev[k][MEASURES.index(measure)]) for measure in MEASURES] for k in range(K)]

In [ ]:
[[write_results_file("results/item_weighted-vot_12.csv", metrics_results[k][MEASURES.index(measure)], range_K[k], measure) for measure in MEASURES] for k in range(K)]